# AQI pre-processing

At over 5 million rows, the air quality index dataset is too large to upload to github natively. In this notebook we chop up the country-level data into state-level files.

The original file can be downloaded [from kaggle](https://www.kaggle.com/datasets/calebreigada/us-air-quality-1980present).

---

#### Imports & intro

Below we import pandas, os, and the data, and we preliminarily explore the problematically large csv:

In [1]:
import os
import pandas as pd

aqi = pd.read_csv('../../Data/raw/US_AQI.csv')
aqi.shape

(5617325, 15)

The AQI dataset has a state ID column. There are no missing or null values in `state_id`, and there are 52 different state ID codes, one for each of the 50 states + Puerto Rico + the District of Columbia:

In [2]:
aqi['state_id'].isnull().sum()

0

In [3]:
len(set(aqi['state_id']))

52

In [9]:
len(set(aqi['CBSA Code']))

671

In [10]:
aqi['CBSA Code'].value_counts().sort_values(ascending=False).head(3)

36540    15492
40140    15492
19380    15462
Name: CBSA Code, dtype: int64

In [12]:
aqi[aqi['CBSA Code'] == 80389].head()

,Unnamed: 0,CBSA Code,Date,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_id,state_name,lat,lng,population,density,timezone


#### Subdividing into state-level files:

What if we just chopped this up into 52 datasets...?

We need to stay under ~100MB/file~ 50MB/file...

In [4]:
# o.g. file is about 611 MB:
round(os.path.getsize('../../Data/raw/US_AQI.csv') / (1024 * 1024), 2)

611.27

In [17]:
# make set of ID codes:
state_id_codes = set(aqi['state_id'])

# dictionary of ID codes - dataframe
# using a dictionary is convenient for the file export later
state_id_dataframes = {
    state_id: aqi[aqi['state_id'] == state_id] for state_id in state_id_codes
}

# make subdirectory if it doesn't exist
os.makedirs('aqi_by_state/', exist_ok=True)

# iterate over ID code set and use string dict key as file name to export:
for state_id in state_id_codes:
    state_id_dataframes[state_id].to_csv(
        f'aqi_by_state/{state_id}.csv',
        index=False,
        compression='gzip'
    )

California was too big without compression, but compressing brings the filesize down -- down enough I wonder if just uploading a zip-compressed version of the original file might work...

In [11]:
# subdirectory filepath
filepath = 'aqi_by_state/'

# just printin' there's only 52 of 'em after all:
for file in os.listdir(filepath):
    filesize = os.path.getsize(filepath + file) / (1024 * 1024)
    print(f'{file}: {round(filesize, 2)}')

AK.csv: 0.27
AL.csv: 0.85
AR.csv: 0.46
AZ.csv: 0.81
CA.csv: 3.48
CO.csv: 1.02
CT.csv: 0.49
DC.csv: 0.12
DE.csv: 0.08
FL.csv: 1.64
GA.csv: 0.91
HI.csv: 0.17
IA.csv: 0.79
ID.csv: 0.45
IL.csv: 1.02
IN.csv: 1.21
KS.csv: 0.3
KY.csv: 0.71
LA.csv: 0.89
MA.csv: 0.45
MD.csv: 0.36
ME.csv: 0.38
MI.csv: 1.15
MN.csv: 0.56
MO.csv: 0.57
MS.csv: 0.55
MT.csv: 0.63
NC.csv: 1.34
ND.csv: 0.29
NE.csv: 0.3
NH.csv: 0.44
NJ.csv: 0.33
NM.csv: 0.81
NV.csv: 0.55
NY.csv: 1.12
OH.csv: 1.66
OK.csv: 0.7
OR.csv: 1.06
PA.csv: 1.99
PR.csv: 0.2
RI.csv: 0.12
SC.csv: 0.88
SD.csv: 0.28
TN.csv: 1.13
TX.csv: 1.7
UT.csv: 0.54
VA.csv: 0.63
VT.csv: 0.2
WA.csv: 1.27
WI.csv: 1.15
WV.csv: 0.63
WY.csv: 0.56


In [12]:
sum([os.path.getsize(filepath + file) for file in os.listdir(filepath)]) / (1024 * 1024)

40.20291233062744

In [13]:
aqi.to_csv('aqi-zip.csv', index=False, compression='gzip')

In [14]:
os.path.getsize('aqi-zip.csv') / (1024 * 1024)

40.77420425415039

In [16]:
pd.read_csv('aqi-zip.csv', compression='gzip')

,Unnamed: 0,CBSA Code,Date,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_id,state_name,lat,lng,population,density,timezone
0,0,10140,2022-01-01,21,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
1,1,10140,2022-01-02,12,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
2,2,10140,2022-01-03,18,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
3,3,10140,2022-01-04,19,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
4,4,10140,2022-01-05,17,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5617320,5718366,49740,1980-12-27,52,Moderate,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix
5617321,5718367,49740,1980-12-28,52,Moderate,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix
5617322,5718368,49740,1980-12-29,24,Good,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix
5617323,5718369,49740,1980-12-30,14,Good,CO,1,Yuma,AZ,Arizona,32.5995,-114.5491,137612.0,311.0,America/Phoenix


In [7]:
# move the folder to its new location
os.rename('aqi_by_state/', '../../Data/raw/aqi_by_state/')